Validate modle code

In [13]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import numpy as np


def weird_division(n, d):
    return n / d if d else 0


def validate_model(model, X, Y, fold):
    """
    validates the model with a k-fold validation which is iterated
    returns the mean accuracy, specificiy, recall, precision, f1 score and auc score
    """

    splits = 5
    iteration = 10

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = []
    f1_list = []

    if fold == "Strat":
        folds = StratifiedKFold(n_splits=splits)
    elif fold == "K":
        folds = KFold(splits, shuffle=True)

    # Iterate "interation" times of k-fold
    for i in range(1, iteration):
        # print(f'Iteration {i}/{iteration}')

        acc_total = 0
        specificity_total = 0
        recall_total = 0
        precision_total = 0
        f1_total = 0

        for train_index, test_index in folds.split(X, Y):
            x_train = X.iloc[train_index, :]
            x_test = X.iloc[test_index, :]
            y_train = Y.iloc[train_index, :]
            y_test = Y.iloc[test_index, :]

            # scale
            sc = MinMaxScaler()
            x_train = sc.fit_transform(x_train)
            x_test = sc.transform(x_test)

            # fit model and predict
            model.fit(x_train, np.ravel(y_train))
            y_pred = model.predict(x_test)

            # Logic to see if we need to group by ID 

            conf_matrix = confusion_matrix(y_test, y_pred)
            TN = conf_matrix[0][0]
            FP = conf_matrix[0][1]
            FN = conf_matrix[1][0]
            TP = conf_matrix[1][1]

            accuracy = (weird_division((TP + TN), (TP + TN + FP + FN))) * 100
            recall = weird_division(TP, (TP + FN)) * 100  # recall
            specificity = weird_division(TN, (TN + FP)) * 100
            precision = weird_division(TP, (TP + FP)) * 100
            f1_score = weird_division(2 * (recall * precision), (recall + precision))

            # sum it up
            acc_total += accuracy
            recall_total += recall
            specificity_total += specificity
            precision_total += precision
            f1_total += f1_score

        # avg
        accuracy_mean = acc_total / splits
        recall_mean = recall_total / splits
        specificity_mean = specificity_total / splits
        precision_mean = precision_total / splits
        f1_mean = f1_total / splits

        acc_list.append(accuracy_mean)
        recall_list.append(recall_mean)
        specificity_list.append(specificity_mean)
        precision_list.append(precision_mean)
        f1_list.append(f1_mean)

    # print("Accuracy for the 10 iterations: ",  acc_list) #mean accuracy acros the 6 folds for each iteration
    # print("Recall for the 10 iterations: ",  recall_list) #mean accuracy acros the 6 folds for each iteration
    # print("Specificity for the 10 iterations: ",  specificity_list)
    # print("Precision for the 10 iterations: ",  precision_list)
    # print("F1  score for the 10 iterations: ",  f1_list)

    return (
        np.mean(acc_list),
        np.mean(specificity_list),
        np.mean(recall_list),
        np.mean(precision_list),
        np.mean(f1_list),
        # np.mean(auc_list),
    )


Evaluate modle code

In [14]:
def evaluate_model(model, x_train, x_test, y_train, y_test):
    sc = MinMaxScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    accuracy = ((TP + TN) / (TP + TN + FP + FN)) * 100
    recall = (TP / (TP + FN)) * 100  # recall
    specificity = (TN / (TN + FP)) * 100
    precision = (TP / (TP + FP)) * 100
    f1_score = 2 * (recall * precision) / (recall + precision)

    return accuracy, recall, specificity, precision, f1_score

reduce

In [15]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (
    StandardScaler,
    LinearDiscriminantAnalysis as LDA,
)
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from mrmr import mrmr_classif
from sklearn.preprocessing import MinMaxScaler


def feature_reduction_pca(X, variance: float):
    # Normalise:
    x_val = X.values
    x_val = StandardScaler().fit_transform(x_val)

    feat_cols = ["feature" + str(i) for i in range(x_val.shape[1])]

    pca = PCA(n_components=variance)

    principal_components = pca.fit_transform(x_val)

    n_components = principal_components.shape[1]
    print(f"For {variance} variance  ->  {n_components} components were computed")

    col_names = []
    for i in range(n_components):
        col_names.append("PC" + str(i + 1))

    principal_components_df = pd.DataFrame(data=principal_components, columns=col_names)

    return principal_components_df


def feature_reduction_lda(X_train, y_train):
    """
    LDA is supervised so we need a test and train split
    """

    # Normalise:
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)

    # LDA
    lda = LDA(n_components=1)
    X_train = lda.fit_transform(X_train, y_train)

    return X_train


def feature_reduction_mrmr(X, Y, n_components):
    selected_components = mrmr_classif(X=X, y=Y, K=n_components)
    x_train = pd.DataFrame(X).loc[:, selected_components]
    x_train = MinMaxScaler().fit_transform(x_train.values)
    return x_train


Model

In [17]:
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC  
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
path = '/Users/athena.kam/Documents/Thesis/codebase/thesis-2023-athena'
os.chdir(path)

CV_SPLIT = 5

def get_best_param(x_train,y_train):
    param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
    grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=0,return_train_score=True,cv = CV_SPLIT)
    #print(grid.cv_results_)
    grid.fit(x_train,y_train)
    print(grid.best_estimator_.get_params())
    return grid

def train_test_SVC(filename:str,hold_out:bool,include_personal_q:bool, grid_search:bool,reduce:bool,over_sample:bool = False,model_weights:dict = {},random_state:int = 0):

    df = pd.read_csv(filename)

    if include_personal_q:
        df = df[df['noPersonalQ']!=1].reset_index(drop=True)
    else:
        df = df[df['personalQ']!=1].reset_index(drop=True)
    
    headers = df.columns
    non_embeddings_headers = []
    
    for header in headers:
        if header.find('embbedings')<0:
            non_embeddings_headers.append(header)

    X = df.drop(columns=non_embeddings_headers)
    Y = df['classification']
    X = MinMaxScaler().fit_transform(X)
   
    # Test Train split
    if hold_out:
        x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.20,random_state=random_state)

        # Oversample on training set
        if over_sample:
            sm = SMOTE(random_state=12)
            x_train,y_train = sm.fit_resample(x_train, y_train) 

        display(x_train)
    else:
        x_train = X
        y_train = Y

        # Feature Reduction 
    if reduce:
        #x_train = feature_reduction_pca(x_train,0.9).values
        #X = feature_reduction_lda(X,Y)
        X = feature_reduction_mrmr(x_train,x_test,y_train,20)
        
    else:
        x_val = X.values
        X = StandardScaler().fit_transform(x_val)


    if grid_search: 
        grid = get_best_param(x_train=x_train,y_train=y_train) 
        model_svc = grid.best_estimator_
    else:
        model_svc = SVC(C = model_weights['C'],gamma=model_weights['gamma'],kernel=model_weights['kernel'])
        
    accuracy, specificiy, recall, precision, f1_score =validate_model(model_svc,pd.DataFrame(x_train),pd.DataFrame(y_train),"Strat")
    print(f'\tAverage Accuracy: {accuracy} \n\
      Average Specificity: {specificiy} \n\
      Average Recall: {recall}\n\
      Average Precision:{precision}\n\
      Average F1 score {f1_score}\n\
      ')
    
    if hold_out:
        accuracy, specificiy, recall, precision, f1_score =evaluate_model(model_svc,x_train,x_test,y_train,y_test)
        print('____________________________________________')
        print('Evaluate model')
        print(f'\tAccuracy: {accuracy} \n\
        Specificity: {specificiy} \n\
        Recall: {recall}\n\
        Precision:{precision}\n\
        F1 score {f1_score}\n\
        ')
        return accuracy, specificiy, recall, precision, f1_score
    else: 
        return accuracy, specificiy, recall, precision, f1_score


Testing different seeds (Hold out only)

In [20]:
def train_test_SVC_TEST_avg_seeds(
    filename: str,
    hold_out: bool = True,
    include_personal_q: bool = False,
    grid_search: bool = True,
    reduce: bool = True,
    model_weights: dict = {},
    over_sample: bool = False,
):
    random_states = [0, 5, 13, 27, 36, 42]
    n_states = len(random_states)

    acc_list = []
    specificity_list = []
    recall_list = []
    precision_list = []
    f1_list = []

    for random_state in random_states:
        print(f"Random State: {random_state}")
        accuracy, specificiy, recall, precision, f1_score = train_test_SVC(
            filename=filename,
            hold_out=hold_out,
            include_personal_q=include_personal_q,
            grid_search=grid_search,
            reduce=reduce,
            model_weights=model_weights,
            random_state=random_state,
            over_sample=over_sample
        )
        acc_list.append(accuracy)
        specificity_list.append(specificiy)
        recall_list.append(recall)
        precision_list.append(precision)
        f1_list.append(f1_score)

    accuracy_mean = np.mean(acc_list)
    recall_mean = np.mean(recall_list)
    specificity_mean = np.mean(specificity_list)
    precision_mean = np.mean(precision_list)
    f1_mean = np.mean(f1_list)

    print(
        "Accuracy list: ", acc_list
    )  # mean accuracy acros the 6 folds for each iteration
    print(
        "Recall list: ", recall_list
    )  # mean accuracy acros the 6 folds for each iteration
    print("Specificity list: ", specificity_list)
    print("Precision list: ", precision_list)
    print("F1  score list: ", f1_list)

    print(
        f"\tAverage Accuracy: {accuracy_mean} \n\
      Average Specificity: {recall_mean} \n\
      Average Recall: {specificity_mean}\n\
      Average Precision:{precision_mean}\n\
      Average F1 score {f1_mean}\n\
      "
    )

In [21]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_bert_sentence_embeddings_transformed.csv',False,False,True,reduce=True)

UnboundLocalError: local variable 'x_test' referenced before assignment

In [12]:
train_test_SVC('datasets/transformed/spontaneousDialogueOnly_google_bert_sentence_embeddings_transformed.csv', hold_out=True,include_personal_q=False,grid_search=True,reduce=True,over_sample=True)

100%|██████████| 20/20 [00:02<00:00,  8.49it/s]


array([[0.63918721, 0.42318996, 0.33984537, ..., 0.34883306, 0.42704771,
        0.61731088],
       [0.58296712, 0.77668962, 0.54420001, ..., 0.36025917, 0.34743428,
        0.42476268],
       [0.68747084, 0.65538472, 0.68281257, ..., 0.4426189 , 0.49713246,
        0.76518929],
       ...,
       [0.62945962, 0.39481075, 0.55199774, ..., 0.42129623, 0.42903228,
        0.33697322],
       [0.51552197, 0.43719799, 0.68974262, ..., 0.43212411, 0.59278877,
        0.1025622 ],
       [0.50919532, 0.27386276, 0.5849214 , ..., 0.50509131, 0.58784281,
        0.70425268]])

{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 77.3306182895224 
      Average Specificity: 72.71771771771773 
      Average Recall: 81.92192192192192
      Average Precision:75.28401663695782
      Average F1 score 78.04672955629597
      
____________________________________________
Evaluate model
	Accuracy: 63.1578947368421 
        Specificity: 53.57142857142857 
        Recall: 68.75
        Precision:50.0
        F1 score 51.724137931034484
        


(63.1578947368421, 53.57142857142857, 68.75, 50.0, 51.724137931034484)

CV score is none

In [66]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=True)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  9.35it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 100.0 
      Average Recall: 95.0
      Average Precision:100.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 75.0 
        Specificity: 66.66666666666666 
        Recall: 80.0
        Precision:66.66666666666666
        F1 score 66.66666666666666
        
Random State: 5


100%|██████████| 20/20 [00:02<00:00,  8.82it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 81.42857142857142 
      Average Specificity: 65.0 
      Average Recall: 100.0
      Average Precision:77.0
      Average F1 score 85.47619047619048
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 93.33333333333334 
      Average Specificity: 86.66666666666666 
      Average Recall: 100.0
      Average Precision:90.0
      Average F1 score 94.28571428571429
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 36


100%|██████████| 20/20 [00:01<00:00, 10.48it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 95.0 
      Average Recall: 100.0
      Average Precision:95.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 62.5 
        Specificity: 33.33333333333333 
        Recall: 80.0
        Precision:50.0
        F1 score 40.0
        
Random State: 42


100%|██████████| 20/20 [00:01<00:00, 10.33it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 76.66666666666666 
      Average Specificity: 79.99999999999999 
      Average Recall: 73.33333333333333
      Average Precision:85.33333333333333
      Average F1 score 74.33333333333333
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 100.0 
        Recall: 85.71428571428571
        Precision:50.0
        F1 score 66.66666666666667
        
Accuracy list:  [75.0, 100.0, 75.0, 100.0, 62.5, 87.5]
Recall list:  [80.0, 100.0, 100.0, 100.0, 80.0, 85.71428571428571]
Specificity list:  [66.66666666666666, 100.0, 71.42857142857143, 100.0, 33.33333333333333, 100.0]
Precision list:  [66.66666666666666, 100.0, 100.0, 100.0, 50.0, 50.0]
F1  score list: 

CV F1


In [64]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=True)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  8.83it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 100.0 
      Average Recall: 95.0
      Average Precision:100.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 75.0 
        Specificity: 66.66666666666666 
        Recall: 80.0
        Precision:66.66666666666666
        F1 score 66.66666666666666
        
Random State: 5


100%|██████████| 20/20 [00:01<00:00, 10.39it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'poly', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 81.42857142857142 
      Average Specificity: 65.0 
      Average Recall: 100.0
      Average Precision:77.0
      Average F1 score 85.47619047619048
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 93.33333333333334 
      Average Specificity: 86.66666666666666 
      Average Recall: 100.0
      Average Precision:90.0
      Average F1 score 94.28571428571429
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 36


100%|██████████| 20/20 [00:02<00:00,  9.97it/s]


{'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 95.0 
      Average Recall: 100.0
      Average Precision:95.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 62.5 
        Specificity: 33.33333333333333 
        Recall: 80.0
        Precision:50.0
        F1 score 40.0
        
Random State: 42


100%|██████████| 20/20 [00:02<00:00,  8.83it/s]


{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 76.66666666666666 
      Average Specificity: 79.99999999999999 
      Average Recall: 73.33333333333333
      Average Precision:85.33333333333333
      Average F1 score 74.33333333333333
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 100.0 
        Recall: 85.71428571428571
        Precision:50.0
        F1 score 66.66666666666667
        
Accuracy list:  [75.0, 100.0, 62.5, 100.0, 62.5, 87.5]
Recall list:  [80.0, 100.0, 100.0, 100.0, 80.0, 85.71428571428571]
Specificity list:  [66.66666666666666, 100.0, 57.14285714285714, 100.0, 33.33333333333333, 100.0]
Precision list:  [66.66666666666666, 100.0, 100.0, 100.0, 50.0, 50.0]
F1  score list: 

MINMAXSCALE

In [13]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=True)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  9.06it/s]


{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 96.66666666666667 
      Average Specificity: 100.0 
      Average Recall: 93.33333333333333
      Average Precision:100.0
      Average F1 score 96.0
      
____________________________________________
Evaluate model
	Accuracy: 63.63636363636363 
        Specificity: 40.0 
        Recall: 83.33333333333334
        Precision:66.66666666666666
        F1 score 49.99999999999999
        
Random State: 5


100%|██████████| 20/20 [00:02<00:00,  9.10it/s]


{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 86.0 
      Average Specificity: 70.0 
      Average Recall: 100.0
      Average Precision:82.0
      Average F1 score 89.28571428571429
      
____________________________________________
Evaluate model
	Accuracy: 90.9090909090909 
        Specificity: 100.0 
        Recall: 87.5
        Precision:75.0
        F1 score 85.71428571428571
        
Random State: 13


...

{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 93.33333333333334 
      Average Specificity: 93.33333333333333 
      Average Recall: 93.33333333333333
      Average Precision:95.0
      Average F1 score 93.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 36


100%|██████████| 20/20 [00:01<00:00, 10.06it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 94.28571428571429 
      Average Specificity: 90.0 
      Average Recall: 100.0
      Average Precision:92.0
      Average F1 score 95.0
      
____________________________________________
Evaluate model
	Accuracy: 81.81818181818183 
        Specificity: 80.0 
        Recall: 83.33333333333334
        Precision:80.0
        F1 score 80.0
        
Random State: 42


100%|██████████| 20/20 [00:01<00:00, 10.03it/s]


{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 76.0 
      Average Specificity: 93.33333333333333 
      Average Recall: 56.666666666666664
      Average Precision:90.0
      Average F1 score 67.33333333333334
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Accuracy list:  [63.63636363636363, 90.9090909090909, 81.81818181818183, 100.0, 81.81818181818183, 100.0]
Recall list:  [83.33333333333334, 87.5, 66.66666666666666, 100.0, 83.33333333333334, 100.0]
Specificity list:  [40.0, 100.0, 87.5, 100.0, 80.0, 100.0]
Precision list:  [66.66666666666666, 75.0, 87.5, 100.0, 80.0, 100.0]
F1  score list:  [49.99999999

No scale

In [10]:
train_test_SVC_TEST_avg_seeds('datasets/transformed/spontaneousDialogueOnly_google_bert_embeddings_transformed.csv',over_sample=True)

Random State: 0


100%|██████████| 20/20 [00:02<00:00,  9.12it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 100.0 
      Average Specificity: 100.0 
      Average Recall: 100.0
      Average Precision:100.0
      Average F1 score 100.0
      
____________________________________________
Evaluate model
	Accuracy: 75.0 
        Specificity: 100.0 
        Recall: 60.0
        Precision:60.0
        F1 score 75.0
        
Random State: 5


100%|██████████| 20/20 [00:02<00:00,  8.43it/s]


{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 83.80952380952381 
      Average Specificity: 81.66666666666666 
      Average Recall: 86.66666666666666
      Average Precision:87.0
      Average F1 score 84.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 100.0 
        Specificity: 100.0 
        Recall: 100.0
        Precision:100.0
        F1 score 100.0
        
Random State: 13


...

{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 93.80952380952381 
      Average Specificity: 93.33333333333333 
      Average Recall: 95.0
      Average Precision:95.0
      Average F1 score 94.28571428571429
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 50.0 
        Recall: 100.0
        Precision:100.0
        F1 score 66.66666666666667
        
Random State: 36


100%|██████████| 20/20 [00:02<00:00,  8.31it/s]


{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 97.14285714285714 
      Average Specificity: 95.0 
      Average Recall: 100.0
      Average Precision:95.0
      Average F1 score 97.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 100.0 
        Recall: 80.0
        Precision:75.0
        F1 score 85.71428571428571
        
Random State: 42


100%|██████████| 20/20 [00:02<00:00,  8.94it/s]


{'C': 1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
	Average Accuracy: 93.33333333333334 
      Average Specificity: 93.33333333333333 
      Average Recall: 93.33333333333333
      Average Precision:95.0
      Average F1 score 93.14285714285714
      
____________________________________________
Evaluate model
	Accuracy: 87.5 
        Specificity: 100.0 
        Recall: 85.71428571428571
        Precision:50.0
        F1 score 66.66666666666667
        
Accuracy list:  [75.0, 100.0, 100.0, 87.5, 87.5, 87.5]
Recall list:  [60.0, 100.0, 100.0, 100.0, 80.0, 85.71428571428571]
Specificity list:  [100.0, 100.0, 100.0, 50.0, 100.0, 100.0]
Precision list:  [60.0, 100.0, 100.0, 100.0, 75.0, 50.0]
F1  score list:  [75.0, 100.0, 100.0, 66.66666666666667, 85.71428571428571, 66.